In principle, the Law of the Wall only applies when the boundary layer is neutrally stable (Richardson number ~ 0).

In [93]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import choice, normal

from scipy.signal import periodogram, correlate, convolve
from scipy.optimize import curve_fit

from glob import glob

from datetime import datetime

from local_utils import *

aspect_ratio = 16./9 # On HD projectors nowadays, this is the aspect ratio.
                     # so I make my figures using that ratio so they show up nicely in presentations.
BoiseState_blue = "#0033A0"
BoiseState_orange = "#D64309"

# Von Karman's constant
kappa = 0.4
g = 9.81 # m/s^2
heights = {"48": 62, "49": 85, "50": 108, "51": 154}

def bulk_Richardson(Delta_z, Delta_T, T, u, cp=1000., g=9.81):
    # MKS units!
    # g in m/s^2
    # cp in J/kg - https://www.ohio.edu/mechanical/thermo/property_tables/air/air_Cp_Cv.html
    
    return g*(Delta_T + g/cp)*Delta_z/(T*u*u)

def calc_x(Ri):
    return (1. - 16*Ri)**(1./4)

def calc_phi(Ri):
    return 1./calc_x(Ri)

def calc_Psi(x):
    return 2*np.log((1 + x)/2.) + np.log((1 + x**2)/2.) - 2*np.arctan(x) + np.pi/2.

def calc_Ri(Rb, z_over_zstar=heights["48"]/0.01):

    x = calc_x(Rb)
    phi = calc_phi(Rb)
    Psi = calc_Psi(x)
    
    return Rb*((np.log(z_over_zstar) - Psi)/phi)**2

In [94]:
ls = glob("Field_Experiments/Drone_Experiments/Seal Beach*.csv")

data = {}
for i in range(len(ls)):
    key = ls[i].split("26368")[1][0:2]
    data[key] = np.genfromtxt(ls[i], skip_header=10, delimiter=',')
    
# Time	Temp	Wet Bulb Temp.	Rel. Hum.	Baro.	Altitude	Station P.	Wind Speed	Heat Index	Dew Point	Dens. Alt.	Crosswind	Headwind	Mag. Dir.	True Dir.	Wind Chill    
Delta_z = (heights["51"] - heights["48"])/100. # convert to m
Delta_T = np.mean(data["51"][:,1] - data["48"][:,1])
T = np.mean([data["51"][:,1], data["48"][:,1]])
u = np.mean([data["51"][:,7], data["48"][:,7]])

Rb = bulk_Richardson(Delta_z, Delta_T, T, u)
print(bulk_Richardson(Delta_z, Delta_T, T, u))
# -0.007502600129211185

-0.007502600129211185


In [96]:
# And then looking at equations from Greeley+ (1995)
x = calc_x(Rb)
phi = calc_phi(Rb)
Psi = calc_Psi(x)
print(Psi)
# 0.028947618903049976 - very small compared to log(z/z_star)
#
# Compared to ln(z/z_star)
zstar = 0.01 # See Experimenting with roughness scale measurements.ipynb
print(np.log(heights["48"]/zstar))
# 8.732304571033183

print(Psi/np.log(heights["48"]/zstar))

0.028947618903049976
8.732304571033183
0.0033150033496398045
